# ADM: Homework 3 Group #24

_Pol Ribó León_ , _Reza Pourrahim_, _Federica Spoto_ 

The goal of this homework is to create a search engine. We are going to analyze the Airbnb property listings in Texas, collected in the file **"Airbnb_Texas_Rentals.csv"** .

# Step 1: Data

### Importing libraries and data

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
from nltk.stem import PorterStemmer
st = PorterStemmer()
import math
import json
from scipy import spatial
import operator
import heapq 
import folium
from geopy import distance
pd.set_option('display.max_colwidth',-1)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bchdnl63l42f205d\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv("Airbnb_Texas_Rentals.csv", sep=',', encoding='latin1')

In [3]:
len(data)

18259

# Step 2: Create documents

### Creating the .tsv files

In [4]:
for i in range(len(data)):
    record_file= open(r'C:/Users/bchdnl63l42f205d/Desktop/Magistrale/Primo anno/AMD/HW 3/try/doc_'+str(i)+'.tsv', "w", encoding="utf-8") 
    for j in range(10):            
        record_file.write('%s\t' %str(data.iloc[i, j]))
    record_file.close()

# Step 3: Search Engine

## 3.1) Conjunctive query

### 3.1.1) Create your index!

#### Creating the dictionary

In [5]:
Dict={}
Inv_Dict={}
for doc in range(len(data)):
    f = open("C:/Users/bchdnl63l42f205d/Desktop/Magistrale/Primo anno/AMD/HW 3/try/doc_"+str(doc)+".tsv", 'r', encoding='utf-8')
    for line in f:
        ou = line.strip().split('\t')
        break
    s=[]
    # Cleaning the data: removing punctuation and stopwords, then stemming
    for i in ou:
        i=re.sub(r'[^\w\s]','',i)
        s.append(i)
    s=s[5]+" "+s[8]
    s=s.lower()
    s2=" ".join(x for x in s.split() if x not in stop)
    s3=" ".join([st.stem(word) for word in s2.split()])
    
    
    for i in set(s3.split()):
        if i not in Dict:
            Inv_Dict[len(Dict)]=["doc_"+str(doc)]
            Dict[i]=len(Dict)        
        else:
            Inv_Dict[Dict.get(i)].append("doc_"+str(doc))




The first dictionary contains all the words of the documents and assigns them a _term id_.

In [6]:
dict(list(Dict.items())[0:2]) # Checking the first terms

{'avail': 0, 'queen': 1}

In the second one, for each _term id_ there is the list of documents in which it appears.

In [7]:
dict(list(Inv_Dict.items())[0:1]) # Checking the first terms

{0: ['doc_0',
  'doc_6',
  'doc_7',
  'doc_14',
  'doc_25',
  'doc_31',
  'doc_38',
  'doc_39',
  'doc_43',
  'doc_58',
  'doc_76',
  'doc_79',
  'doc_85',
  'doc_103',
  'doc_106',
  'doc_110',
  'doc_120',
  'doc_129',
  'doc_131',
  'doc_134',
  'doc_179',
  'doc_188',
  'doc_220',
  'doc_270',
  'doc_278',
  'doc_284',
  'doc_329',
  'doc_339',
  'doc_376',
  'doc_381',
  'doc_386',
  'doc_401',
  'doc_403',
  'doc_445',
  'doc_489',
  'doc_496',
  'doc_510',
  'doc_515',
  'doc_519',
  'doc_525',
  'doc_573',
  'doc_577',
  'doc_581',
  'doc_582',
  'doc_598',
  'doc_601',
  'doc_616',
  'doc_627',
  'doc_631',
  'doc_632',
  'doc_642',
  'doc_645',
  'doc_646',
  'doc_687',
  'doc_693',
  'doc_708',
  'doc_710',
  'doc_729',
  'doc_730',
  'doc_736',
  'doc_786',
  'doc_790',
  'doc_803',
  'doc_837',
  'doc_860',
  'doc_861',
  'doc_862',
  'doc_863',
  'doc_868',
  'doc_873',
  'doc_887',
  'doc_898',
  'doc_906',
  'doc_918',
  'doc_926',
  'doc_955',
  'doc_962',
  'doc_991',

#### Saving the two dictionaries

In [8]:
with open("C:/Users/bchdnl63l42f205d/Desktop/Dict.txt",'w') as file:
    file.write(json.dumps(Dict))

with open("C:/Users/bchdnl63l42f205d/Desktop/Inv_Dict.txt",'w') as file:
    file.write(json.dumps(Inv_Dict))


#### Creating a function to build the search engine

In [9]:
def queryprocess(query):
    
    # cleaning the text of the query
    query=(re.sub(r'[^\w\s]','',query)).lower()
    query2=" ".join(x for x in query.split() if x not in stop)
    query3=" ".join([st.stem(word) for word in query2.split()])
    
    # Taking the term_id of the query's words
    qlist=query3.split()
    index=[]
    for word in qlist:
        index.append(Dict.get(word))
    
    # Finding the documents with the words of the query
    docs=[]
    for number in index:
        docs.append(Inv_Dict.get(number))
    
    # Selecting only the documents containing ALL the words in the query: we take for each term_id the list of documents 
    # containing that word and append it in another list, so we'll obtain a nested list with the list of documents that 
    # contain the word. After, we store all the documents in a single list (lis).
    lis=[]
    for item in docs:
        for i in item:
            lis.append(i)
    # Then we create a dictionary to store how many words of the query the document has.
    count=dict(Counter(lis))
    
    # Eventually we select only the documents that contain all the words of the query.
    outputlist=[]
    for i,j in count.items():
        if j==len(qlist):
            outputlist.append(i)    
            
    # Returning the result
    df=pd.DataFrame()
    for doc in outputlist:
            f = open("C:/Users/bchdnl63l42f205d/Desktop/Magistrale/Primo anno/AMD/HW 3/try/"+doc+".tsv", 'r', encoding='utf-8')
            for line in f:
                ou = line.strip().split('\t')

            x=[ou[8],ou[5],ou[3],ou[9]]
            interest=pd.DataFrame([x],columns=["Title","Description","City","Url"])
            df=pd.concat([df,interest])

    display(df)

### 3.1.2) Execute the query

After the building of the Search Engine we procede testing it: we give a request as an input and check if the function built returns announcement which satisfy the query.

In [10]:
query=input("Query: ")
queryprocess(query)

Query: house near the grocery


,Title,Description,City,Url
0,(B) Private Room Gated Community Near DFW Airport,"Bookings available from Aug 19th. You are booking a modern private room in a two story house. Located near major highways, restaurants, grocery stores, Texas star golf course, parks, AT&amp;T stadium, and just 10 min to DFW airport. My space is in between Dallas and Fort Worth. Hence, great for interns, travel workers, students (15 min to UT Arlington), business travelers, and even for adventurers. Short-term and Long-term discounts.",Euless,https://www.airbnb.com/rooms/17806073?location=Colleyville%2C%20TX
0,Spacious home near Sea World and Lackland AFB,"Enjoy three large bedrooms and a loft. Entire house is for rent (3160 sq ft). Sleeps 8 comfortable. One king size bed, one queen bed, one double bed, and day bed (two twin beds). Full laundry. Near Air Force Base, Sea World (3 mi) and Six Flags and Riverwalk. Enjoy a beautiful living room, kitchen, pool table, and patio. Plenty of room for a big family or group. Fresh towels and linens. Nearby shopping, restaurants, and grocery store. Fully furnished kitchen. Safe and quiet neighborhood.",San Antonio,https://www.airbnb.com/rooms/16417007?location=Castroville%2C%20TX
0,Spacious Living Near Lake Houston,"My house is in a Master-Planned community w/lots of green space, parks, playgrounds, jogging trail &amp; ponds. The living space is modern, spacious, w/an open floor plan, beautifully furnished w/1 king sized bed &amp; 3 queen sized beds. Close proximity to city attractions including 25 mins from Downtown Houston, 10 mins from Bush intercontinental airport &amp; many energy companies along beltway 8. There are several attractions; restaurants, movie theater, bowling alley, shopping center, &amp; grocery stores.",Humble,https://www.airbnb.com/rooms/19196476?location=Cleveland%2C%20TX
0,(A) Private Room Gated Community Near DFW Airport,"Bookings available from Aug 12th. You are booking a modern private room in a two story house. Located near major highways, restaurants, grocery stores, Texas star golf course, parks, AT&amp;T stadium, and just 10 min to DFW airport. My space is in between Dallas and Fort Worth cities. Great for interns, travel workers, students (15 min to UT Arlington), business travelers, and even for adventurers. Short-term and Long-term discounts.",Euless,https://www.airbnb.com/rooms/17774106?location=Bedford%2C%20TX
0,Two Private Rooms Gated Community Near DFW Airport,"Bookings available from Aug 19th, 2017. You are booking two modern private room's in a two story house. Located near major highways, restaurants, grocery stores, Texas star golf course, parks, AT&amp;T stadium, and just 10 min to DFW airport. My space is in between Dallas and Fort Worth. Hence, great for interns, travel workers, students (15 min to UT Arlington), business travelers, and even for adventurers. Short-term and Long-term discounts.",Euless,https://www.airbnb.com/rooms/18288072?location=Bedford%2C%20TX
0,Private Room Near the Airports,"One of three private rooms in a two story home located in a nice and quiet neighborhood. Close to major highways. Easy access to entire Dallas/Fort Worth area. Very close to DFW airport and not far from Love Field Airport either. Walking distance to a park, a movie theater, many shopping venues (including grocery store and department store) and many restaurants (0.5-1.0 mile). The house is bright and airy with high ceiling. Good Wi-Fi connection. Full kitchen/laundry room access .",Irving,https://www.airbnb.com/rooms/13602580?location=Coppell%2C%20TX
0,Two Private Rooms Gated Community Near DFW Airport,"Bookings available from Aug 19th, 2017. You are booking two modern private room's in a two story house. Located near major highways, restaurants, grocery stores, Texas star golf course, parks, AT&amp;T stadium, and just 10 min to DFW airport. My space is in between Dallas and Fort Worth. Hence, great for interns, travel workers, students (15 min to UT Arlington), business tr

In [11]:
query=input("Query: ")
queryprocess(query)

Query: apartment with pool by the sea


,Title,Description,City,Url
0,"Cool Hill Country Retreat, Pool & Spa at El Cielo","Ready for spacious comfort, starlight and seclusion? Romance?\nOur separate art studio apartment offers that and more: memory foam top queen bed, 20 mile hilltop views, complete kitchen, original art, Netflix TV, private outdoor table plus screen porch and decks to use when you wish. Steps away (behind the main house) --above ground pool and sparkling spa. We've hosted couples, friends and newlyweds. Near Bandera, Boerne, Kerrville, Fredericksburg, San Antonio, wineries, Sea World and more.",Pipe Creek,https://www.airbnb.com/rooms/11278325?location=Center%20Point%2C%20TX
0,Elegant Eilan Luxury Apartment at La Cantera,"La Cantera is the area surrounding Ãilan. It offers easy access to shopping, championship golf courses, Sea World, &amp; Six Flags Fiesta Texas theme park. Also only minutes to downtown San Antonio! There is a full-size front load washer and dryer, gas cook top range, granite counter tops. There are several pools and a hot tub, a sprawling Fitness Center. On location are Ruths Chris Steak House, Piatti Italian Restaurant, Juicessential, Georges Keep Bar, and Sustenio restaurant.",San Antonio,https://www.airbnb.com/rooms/18264474?location=Bulverde%2C%20TX
0,"Cool Hill Country Retreat, Pool & Spa at El Cielo","Ready for spacious comfort, starlight and seclusion? Romance?\nOur separate art studio apartment offers that and more: memory foam top queen bed, 20 mile hilltop views, complete kitchen, original art, Netflix TV, private outdoor table plus screen porch and decks to use when you wish. Steps away (behind the main house) --above ground pool and sparkling spa. We've hosted couples, friends and newlyweds. Near Bandera, Boerne, Kerrville, Fredericksburg, San Antonio, wineries, Sea World and more.",Pipe Creek,https://www.airbnb.com/rooms/11278325?location=Boerne%2C%20TX
0,Great Apartment in The Medical District,"Hello! I'm close to The Hospital District. I'm also close to Fiesta Texas, Sea World and La Cantera. Youâll love the comfy bed and great pool. My place is good for couples, solo adventurers, business travelers and those needing accommodations in the medical district.",San Antonio,https://www.airbnb.com/rooms/14110776?location=Bulverde%2C%20TX
0,Elegant Eilan Luxury Apartment at La Cantera,"La Cantera is the area surrounding Ãilan. It offers easy access to shopping, championship golf courses, Sea World, &amp; Six Flags Fiesta Texas theme park. Also only minutes to downtown San Antonio! There is a full-size front load washer and dryer, gas cook top range, granite counter tops. There are several pools and a hot tub, a sprawling Fitness Center. On location are Ruths Chris Steak House, Piatti Italian Restaurant, Juicessential, Georges Keep Bar, and Sustenio restaurant.",San Antonio,https://www.airbnb.com/rooms/18264474?location=Boerne%2C%20TX
0,Eckhert Apartment,Apartment for 8 people. Everything for a holiday or business stay:\n- Wifi\n- Kitchen\n- Dinning room\n- Living room\n- TV (4)\n- Air conditioner\n- Washing center\n- 1 bedroom (1 king size bed)\n- 1 Bedroom (2 double beds)\n- 1 bedroom (2 single beds)\n\nThe place has:\n- Swimming Pool \n- Elevator\n- Telephone\n\nDistances in car:\n- River Walk: 20 min\n- Alamo Bowl: 20 min\n- Shop's at the Quarry: 15 min\n- Six Flags: 15 min\n- Sea World: 19 min\n- North Star Mall: 17 min\n- San Marcos Outlets: 50 min,San Antonio,https://www.airbnb.com/rooms/18805279?location=Bulverde%2C%20TX


# 3.2) Conjunctive query & Ranking score

### 3.2.1) Inverted index

Now, we want to improve our Search Engine: we want it to return the **top-k** documents related to the query (when the documents that contain all the words of the query are more than k), ranked by their similarity to the user's request. 

First, we calculate the _TfIdf score_ for the words of each document.

#### Calculating the _TfIdf score_

In [12]:
TfIdf_Dict={}
n=len(data)

for doc in range(n):
    base={}
    f = open("C:/Users/bchdnl63l42f205d/Desktop/Magistrale/Primo anno/AMD/HW 3/try/doc_"+str(doc)+".tsv", 'r', encoding='utf-8')
    for line in f:
        ou = line.strip().split('\t')
        break
    s=[]
    # Cleaning the words
    for i in ou:
        i=re.sub(r'[^\w\s]','',i)
        s.append(i)
    s=s[5]+" "+s[8]
    s=s.lower()
    s2=" ".join(x for x in s.split() if x not in stop)
    s3=" ".join([st.stem(word) for word in s2.split()])
    
    # Calculating the TfIdf score
    vec=[]
    for word in s3.split():
        Nj=len(Inv_Dict[(Dict[word])])
        Idfj=math.log10(n/Nj)

        Tfji=s3.count(word)
        TfIdf=Tfji*Idfj
        vec.append(TfIdf)
        base[word]=int(TfIdf)
        
    # Normalizing the vector of TfIdf
    norm=0
    for i in vec:
        norm+=i**2
    norm=np.sqrt(norm)
    
    for word in base.keys():
        base[word]=base[word]/norm     
       
        if Dict[word] not in TfIdf_Dict:
            
            TfIdf_Dict[(Dict[word])]=[("doc_"+str(doc),base[word])]
        else:                              
            TfIdf_Dict[(Dict[word])].append(("doc_"+str(doc),base[word]))
    


Then, we saved in a dictionary the _Idf_ values for all the words (it remains the same for all the documents) to calculate faster the _TdIdf score_ of the query's words. In this case the _Idf score_ is obtained as $Idf_j=log(\frac{n+1}{N_j+1})$, beacuse the query is included in the list of documents.

In [13]:
IDF={}
for word in Dict.keys():
    Nj=len(Inv_Dict[(Dict[word])])
    Idfj=math.log10((n+1)/(Nj+1))
    IDF[Dict[word]]=Idfj
    

#### Creating the second Search Engine

In [14]:
def queryprocess_ranking(query,k):
    query=(re.sub(r'[^\w\s]','',query)).lower()
    query2=" ".join(x for x in query.split() if x not in stop)
    query3=" ".join([st.stem(word) for word in query2.split()])

    qlist=query3.split()

    # TfIdf of the query's words
    Tfidf_q=[]
    for word in qlist:
        Tf=qlist.count(word)/len(qlist)
        tfidf=Tf*IDF[Dict[word]]
        Tfidf_q.append(tfidf)
    norm=np.sqrt(np.sum(Tfidf_q)**2)
    Tfidf_q=Tfidf_q/norm
        
    # Finding the docs with all the words in the query   
    index=[]
    for word in qlist:
        index.append(Dict.get(word))

    docs=[]
    for number in index:
        docs.append(Inv_Dict.get(number))

    lis=[]
    for item in docs:
        for i in item:
            lis.append(i)

    count=dict(Counter(lis))

    outputlist=[]
    for i,j in count.items():
        if j==len(qlist):
            outputlist.append(i)    
    
    # Calculating the similarity: for each document we create a list where we append the TfIdf score of the query's words in 
    # that document.  
    doc_sim={}
    for doc in outputlist:
        q_doc=[]
        for word in qlist:
            term_id=Dict[word]
            for i in range(len(TfIdf_Dict[term_id])):
                if TfIdf_Dict[term_id][i][0]==doc:
                    q_doc.append(TfIdf_Dict[term_id][i][1])
        # Then we calculate the cosine similarity between the vector just created and the TfIdf scores of the query
        cos_sim=1-spatial.distance.cosine(q_doc,Tfidf_q) 
        # After, we save the result for each document in a dictionary
        doc_sim[doc]=cos_sim
   
    # Sorting the results using a heap data structure
    heap=[(value,key) for key,value in doc_sim.items()]
    largest=heapq.nlargest(k,heap)
    sorted_doc=[(key,value) for value,key in largest]
    
    # Selecting the top-k results
    if len(sorted_doc)>k:
        sorted_doc=sorted_doc[:k]


    # Returning the results
    df=pd.DataFrame()
    for doc in sorted_doc:
            f = open("C:/Users/bchdnl63l42f205d/Desktop/Magistrale/Primo anno/AMD/HW 3/try/"+doc[0]+".tsv", 'r', encoding='utf-8')
            for line in f:
                ou = line.strip().split('\t')                
            x=[ou[8],ou[5],ou[3],ou[9],doc_sim[doc[0]]]
            interest=pd.DataFrame([x],columns=["Title","Description","City","Url","Similarity"])
            df=pd.concat([df,interest])

    display(df)

In [15]:
query=input("Query: ")
k=int(input("Maximum number of results: "))
queryprocess_ranking(query,k)

Query: house by the sea with tv
Maximum number of results: 10


,Title,Description,City,Url,Similarity
0,Cozy house near Sea World,"Cozy 2 story house , 3 bedrooms , 2,5 bathrooms , living area with wifi, TV . Fully equipped kitchen , laundry room . Located 3 minutes from Sea World , 15 minutes from Shops at La Cantera, Top Golf and I Fly, 20 minutes from Lackland AFB, 10 minutes from Alamo Ranch Shopping Mall, 10 minutes from Northwest Vista College.",San Antonio,https://www.airbnb.com/rooms/16244535?location=Castroville%2C%20TX,0.983512
0,Sea La Vie Spacious short walk to beach!,"Spacious and airy 3/2 beach house.Large living room with queen futon, twin sleeper chair, and couch. Open fully equipped kitchen and dining area. Washer and dryer, ice maker, 2 smart tv's. Wi-fi. Outdoor shower. Plenty of seating on covered deck. Under house has lighting, picnic table, bar and stools, more seating. Bedroom 1-king. Bedroom 2- queen. Bedroom 3- 2 twins. Short walk to the beach and close to shopping and dining. Perfect for families or group.",Crystal Beach,https://www.airbnb.com/rooms/17191332?location=Anahuac%2C%20TX,0.949063
0,Sea La Vie Spacious short walk to beach!,"Spacious and airy 3/2 beach house.Large living room with queen futon, twin sleeper chair, and couch. Open fully equipped kitchen and dining area. Washer and dryer, ice maker, 2 smart tv's. Wi-fi. Outdoor shower. Plenty of seating on covered deck. Under house has lighting, picnic table, bar and stools, more seating. Bedroom 1-king. Bedroom 2- queen. Bedroom 3- 2 twins. Short walk to the beach and close to shopping and dining. Perfect for families or group.",Crystal Beach,https://www.airbnb.com/rooms/17191332?location=Bolivar%20Peninsula%2C%20TX,0.949063
0,Entire Spacious Home Near Sea World and Lackland,"Cozy 2 story house... 4 bedrooms, 2.5 bathrooms, living area with TV, wifi. Fully equiped kitchen and laundry room. Conveniently located 3 minutes from Sea world , 15 minutes to The Rim and Six Flags, Located blocks away from Lackland AFB. 16 miles from Downtown San Antonio.",San Antonio,https://www.airbnb.com/rooms/18134652?location=Castroville%2C%20TX,0.940144
0,"Cool Hill Country Retreat, Pool & Spa at El Cielo","Ready for spacious comfort, starlight and seclusion? Romance?\nOur separate art studio apartment offers that and more: memory foam top queen bed, 20 mile hilltop views, complete kitchen, original art, Netflix TV, private outdoor table plus screen porch and decks to use when you wish. Steps away (behind the main house) --above ground pool and sparkling spa. We've hosted couples, friends and newlyweds. Near Bandera, Boerne, Kerrville, Fredericksburg, San Antonio, wineries, Sea World and more.",Pipe Creek,https://www.airbnb.com/rooms/11278325?location=Center%20Point%2C%20TX,0.921687
0,Dickinson's Perfect Hideaway!,"Incredible opportunity to vacation in a newer custom home built specifically for a vacation home with 5 bedrooms all with private full bathrooms! Sitting on several acres and directly on the Dickinson Bayou waters, you have land and sea options as to how to spend your days. Set up a volleyball net, horseshoe pit, bocce ball course or organize a game of football on land. Also bring your kayaks, canoes, and fishing gear to take advantage of the convenience of being waterfront. Home has everything needed to make your vacation weekend or extended stay a complete success. Relax on the expansive upper wraparound deck on the house, enjoy the lower built out bar and outdoor TV adjacent to outdoor bathroom with 2 showers, 2 propane grills (one upper level and one lower), and full chef's kitchen complete with 2 full size refrigerators.\r\n**NOTE: Reservations for Spring Break (month of March), Memorial Day weekend, Fourth of July, Labor Day weekend, Thanksgiving, and Christmas, and New Years all require a minimum 4 day reservation. Reservations between June 1st and September 1st all require a minimum 3 day reservation.",Dickinson,https://www.airbnb.com/rooms/19406693?location=Bacliff%2C%20TX,0.921687
0,Dickinson's Perfect H

In [16]:
query=input("Query: ")
k=int(input("Maximum number of results: "))
queryprocess_ranking(query,k)

Query: apartment with garden
Maximum number of results: 10


,Title,Description,City,Url,Similarity
0,Beautiful Apt @ Botanical Gardens,"This beautiful apartment is located in a green and quite area inside Alamo Heights of San Antonio, right behind the Botanical Gardens. Apartment is walking distance to the Zoo, Brackenridge Park, San Antonio Country Club and only 5 mins drive to Downtown. It is remodeled with wood and tile floors, ceiling fans in all rooms, custom all tile shower and custom tub/tile shower, granite counter tops with stainless steel under mount sinks. Outside there are fruit trees and lots of parking space",San Antonio,https://www.airbnb.com/rooms/19494083?location=Converse%2C%20TX,0.999619
0,Greeley Gardens Carriage House,This is a beautiful cozy garage apartment with a large porch overlooking the grounds that make up the Greeley Gardens estate. The Apartment has low ceilings (less than 7ft) and is ideal for travelers looking for a serene indoor/outdoor environment to relax during their travels.,Houston,https://www.airbnb.com/rooms/8301898?location=Bellaire%2C%20TX,0.999619
0,Hyde Park UT 1930s Garden Apartment,"Immaculate 1930s garden apartment in Hyde Park. Newly renovated. Open, airy &amp; bright. French doors open onto garden.\nEasy walking distance to shops, restaurants, groceries &amp; UT. \nMake sure to check out the Guidebook.\n",Austin,https://www.airbnb.com/rooms/1755803?location=Brazos%20River%2C%20TX,0.988476
0,Hyde Park UT 1930s Garden Apartment,"Immaculate 1930s garden apartment in Hyde Park. Newly renovated. Open, airy &amp; bright. French doors open onto garden.\nEasy walking distance to shops, restaurants, groceries &amp; UT. \nMake sure to check out the Guidebook.\n",Austin,https://www.airbnb.com/rooms/1755803?location=Cedar%20Park%2C%20TX,0.988476
0,Hyde Park UT 1930s Garden Apartment,"Immaculate 1930s garden apartment in Hyde Park. Newly renovated. Open, airy &amp; bright. French doors open onto garden.\nEasy walking distance to shops, restaurants, groceries &amp; UT. \nMake sure to check out the Guidebook.\n",Austin,https://www.airbnb.com/rooms/1755803?location=Colorado%20River%2C%20TX,0.988476
0,SURROUND YOURSELF WITH CHARM,"Clean &amp; Comfortable two bedroom apt is great for your stay in Grand Prairie, TX! Located only 15 minutes from downtown Dallas. My apartment is about 10 minutes from DFW airport &amp; The Parks Mall in Arlington. About 15 minutes away from Six Flags over Texas, AT&amp;T stadium, Globe Life stadium (Texas Rangers) and Restaurants such as BJ's, TGI Fridays, Olive Garden etc. I don't have a private parking spot, but there is always plenty of open spaces near my apt.",Grand Prairie,https://www.airbnb.com/rooms/18807470?location=Cedar%20Hill%2C%20TX,0.985621
0,Private home close to galleria,"Nice apartment space for you and your guests . Very cozy and quiet . Big garden tub, w/d in unit , extra couch room and study .Spacious kitchen , clean linens , quiet neighborhood .",Houston,https://www.airbnb.com/rooms/18692663?location=Cinco%20Ranch%2C%20TX,0.985621
0,LOCAL VIBE + Near Downtown Dallas,"Feel like a local! Conveniently located near downtown Dallas. Experience Dallas' some top-rated favorites such as, Garden Cafe, Sugar Bacon, along with many other must-visit places. Only 2.5 miles to downtown. Fresh 1 bdrm / 1 bath apartment, with full kitchen, living &amp; dining areas, accompanied with a quaint courtyard. Direct access to public transportation. Don't have the dates you need? Check out: https://www.airbnb.com/manage-your-space/12274983",Dallas,https://www.airbnb.com/rooms/15172975?location=Carrollton%2C%20TX,0.985621
0,LOCAL VIBE + Near Downtown Dallas,"Feel like a local! Conveniently located near downtown Dallas. Experience Dallas' some top-rated favorites such as, Garden Cafe, Sugar Bacon, along with many other must-visit places. Only 2.5 miles to downtown. Fresh 1 bdrm / 1 bath apartment, with full kitchen, living &amp; dining areas, accompanied with a quaint courtyard. Direct access to public transportation. Don't have 

# Step 4: Define a new score!

In the end, we want to define a new score to rank the results given by our Search Engine. In particular, after knowing in which city the user want to stay, our Search Engine will solt the results depending on their convenience per bedroom, measured by the rate $\frac{average\,rate\,per\,night}{bedrooms\,count}$ .

In [17]:
def queryprocess_list(query):
    # Cleaning the query's words
    query=(re.sub(r'[^\w\s]','',query)).lower()
    query2=" ".join(x for x in query.split() if x not in stop)
    query3=" ".join([st.stem(word) for word in query2.split()])
    
    # Finding the docs with all the words in the query 
    qlist=query3.split()
    index=[]
    for word in qlist:
        index.append(Dict.get(word))

    docs=[]
    for number in index:
        docs.append(Inv_Dict.get(number))

    lis=[]
    for item in docs:
        for i in item:
            lis.append(i)

    count=dict(Counter(lis))

    outputlist=[]
    for i,j in count.items():
        if j==len(qlist):
            outputlist.append(i)    
    return(outputlist)

#### Defining the query process with additional information

In [18]:
def query_newscore(query,city,k):
    # We take the documents that contains all the words of the query using the previous function
    lis=queryprocess_list(query)
    
    # We select the documents where the city match the request one
    city_lis=[]
    for doc in lis:
        f = open("C:/Users/bchdnl63l42f205d/Desktop/Magistrale/Primo anno/AMD/HW 3/try/"+(doc)+".tsv", 'r', encoding='utf-8')
        for line in f:
            ou = line.strip().split('\t')
            break
        if ou[3].lower()==city.lower() :        
            city_lis.append(doc)
    
    # We calcolate the score dividing the price by the bedroom count
    score={}
    for doc in city_lis:
        f = open("C:/Users/bchdnl63l42f205d/Desktop/Magistrale/Primo anno/AMD/HW 3/try/"+(doc)+".tsv", 'r', encoding='utf-8')
        for line in f:
            ou = line.strip().split('\t')
            break
        price=re.sub(r'[^\w\s]','',ou[1])
        bedroom=ou[2]
        if price!='nan' and bedroom!='nan':
            if bedroom=="Studio":
                bedroom=1
            score[doc]=int(price)/int(bedroom)
    
    # We sorted the result from the cheapest to the most expensive 
    heap=[(value,key) for key,value in score.items()]
    smallest=heapq.nsmallest(k,heap)
    sorted_doc=[(key,value) for value,key in smallest]

    # Selecting the top-k results
    if len(sorted_doc)>k:
        sorted_doc=sorted_doc[:k]

    # Returning the results
    df=pd.DataFrame()
    for doc in sorted_doc:
            f = open("C:/Users/bchdnl63l42f205d/Desktop/Magistrale/Primo anno/AMD/HW 3/try/"+doc[0]+".tsv", 'r', encoding='utf-8')
            for line in f:
                ou = line.strip().split('\t')                
            x=[ou[8],ou[5],ou[3],ou[9],round(score[doc[0]],2)]
            interest=pd.DataFrame([x],columns=["Title","Description","City","Url","Price per bedroom"])
            df=pd.concat([df,interest])

    display(df)

In [19]:
query=input("Query: ")
city=input("In what city do you want to stay? ")
k=int(input("Maximum number of results: "))
query_newscore(query,city,k)

Query: house near museum
In what city do you want to stay? san antonio
Maximum number of results: 7


,Title,Description,City,Url,Price per bedroom
0,"Little Blue ""Pearl"" House","Perfect for urban dwellers. Safe neighborhood in transition. Walking distance to restaurants and dining, coffee shops, YMCA and nightlife. Family-friendly activities nearby (Zoo, Brackenridge Park, Children's Museum, Riverwalk by the Pearl district ). Great location between downtown and the airport and near the Pearl district (an easy 1 mile walk away). Good for couples, solo adventurers, business travelers, and small families (with kids). Free, secure parking onsite.",San Antonio,https://www.airbnb.com/rooms/8497305?location=Alamo%20Heights%2C%20TX,57.5
0,Spacious! Upstairs duplex 5 minutes from downtown!,"1920's home turned duplex, in close proximity to Downtown, parks, museums, Zoo, military bases, Pearl, airport and more. A short walk to several restaurants, breakfast tacos too. This is an older house in a historic neighborhood with over 1,000 square feet. I am out of town a lot but my best friends live below and is the caretaker. There are 3 window units, two TVs with cable and TiVo; in living room and bedroom. Bonus: One covered parking space (more street parking near by).",San Antonio,https://www.airbnb.com/rooms/12407501?location=Alamo%20Heights%2C%20TX,62.5
0,Stylish Casita in Chic Neighborhood,"Casita Corazon, in the heart of town is close to restaurants, trendy shops, coffee houses, bakeries, upscale grocery store, parks, museums, public transport, near downtown, the Riverwalk, 10 minutes to airport with easy access to 281, I-37, I-35, I-10, 5 minutes from FedEx Office. You will be be hosted in contemporary, stylish rooms filled with art , large, flat TVs, wifi, work areas, new comfortable beds, 1.5 new baths, multiple beautiful decks, patios , outdoor seating &amp; dining areas.",San Antonio,https://www.airbnb.com/rooms/13992373?location=Alamo%20Heights%2C%20TX,125.0
0,Stylish Casita in Chic Neighborhood,"Casita Corazon, in the heart of town is close to restaurants, trendy shops, coffee houses, bakeries, upscale grocery store, parks, museums, public transport, near downtown, the Riverwalk, 10 minutes to airport with easy access to 281, I-37, I-35, I-10, 5 minutes from FedEx Office. You will be be hosted in contemporary, stylish rooms filled with art , large, flat TVs, wifi, work areas, new comfortable beds, 1.5 new baths, multiple beautiful decks, patios , outdoor seating &amp; dining areas.",San Antonio,https://www.airbnb.com/rooms/13992373?location=Converse%2C%20TX,125.0


# Bonus Step: Make a nice visualization!

In [20]:
Coords={}
Urls={}
Price={}
for doc in range(len(data)):
    f = open("C:/Users/bchdnl63l42f205d/Desktop/Magistrale/Primo anno/AMD/HW 3/try/doc_"+str(doc)+".tsv", 'r', encoding='utf-8')
    for line in f:
        ou = line.strip().split('\t')
        break

    if ou[8] not in Coords:
        if ou[6]!='nan' and ou[7]!='nan':
            Coords[ou[8]]=[float(ou[6]),float(ou[7])]
            Urls[ou[8]]=ou[9]
            Price[ou[8]]=ou[1]

In [21]:
def Folium(inpu,r):
    pos=[float(inpu[0]),float(inpu[1])]
    m = folium.Map(
        location=pos,
        zoom_start=13
    )
    tooltip = 'Click me!'

    folium.Circle(
        radius=r*1000,
        location=pos,
        popup='The Waterfront',
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(m)

    for name in Coords.keys():
        url=Urls[name]
        tool=Price[name]
        if distance.great_circle(pos, Coords[name]).km <= r:
            folium.Marker(Coords[name], popup=folium.Popup('Url: <a href="' +url+' "target="_blank"> '+url+' </a>'), tooltip=tool,icon=folium.Icon(color='blue',icon='home')).add_to(m)
    
    folium.Marker(pos, popup='<b>Searched Position</b>', tooltip=tooltip, icon=folium.Icon(color='green',icon='star')).add_to(m)

    m.save('map.html')

    display(m)

In [24]:
inpu=input("Coordinates: ").split()
r=float(input("Radius in km:")) 
Folium(inpu,r)

Coordinates: 29.50306 -98.44768
Radius in km:10
